<b><u>Observations</b></u>
<p>1. observation</p>
<p>2. observation</p>
<p>3. observation</p>

In [1]:
# Dependencies
import random
import json
import requests
from pprint import pprint
from config import api_key
import pandas as pd
import seaborn as sns
from citipy import citipy

# Dependencies for downloading and uncompressing list of cities from OpenWeatherMap
import urllib.request
import gzip

<h1>Generate City List</h1>

In [5]:
# Create a list of 500 random latitudes and longitudes
# Latitudes range from -90 to +90 and longitutes range from -180 to +180
#total_cities = 10
#latitude_list = [random.randint(-9000,9000)/100 for x in range(0, total_cities)]
#longitude_list = [random.randint(-18000,18000)/100 for x in range(0, total_cities)]

# Create a list of cities and their respective country using citipy
#city_list = [citipy.nearest_city(latitude_list[x], longitude_list[x]).city_name for x in range(0, total_cities)]
#country_list = [citipy.nearest_city(latitude_list[x], longitude_list[x]).country_code for x in range(0, total_cities)]
#print(city_list)
#print(country_list)

In [3]:
# File names
city_index_zip_file = "city.list.json.gz"
city_index_json_file = 'city.list.json'

# Download list of cities and IDs from OpenWeatherMap
urllib.request.urlretrieve ("http://bulk.openweathermap.org/sample/city.list.json.gz", city_index_zip_file)

# Uncompress gzip file to json
inF = gzip.open(city_index_zip_file, 'rb')
outF = open(city_index_json_file, 'wb')
outF.write( inF.read() )
inF.close()
outF.close()

# Read json file into a dataframe
city_index_df = pd.read_json(city_index_json_file)

city_index_df.head()

,coord,country,id,name
0,"{'lon': 34.283333, 'lat': 44.549999}",UA,707860,Hurzuf
1,"{'lon': 37.666668, 'lat': 55.683334}",RU,519188,Novinki
2,"{'lon': 84.633331, 'lat': 28}",NP,1283378,GorkhÄ�
3,"{'lon': 76, 'lat': 29}",IN,1270260,State of HaryÄ�na
4,"{'lon': 33.900002, 'lat': 44.599998}",UA,708546,Holubynka


In [115]:
# Pick 500 random cities
total_cities = 3
city_name_list = []
city_id_list = []
city_country_list = []
city_long_list = []
city_lat_list = []

for index in range(0, total_cities):
    row = random.randint(0, len(city_index_df)-1)
    if city_index_df['id'][row] in city_id_list:
        print('City is already in list')
    else:
        city_name_list.append(city_index_df['name'][row])
        city_id_list.append(city_index_df['id'][row])
        city_country_list.append(city_index_df['country'][row])
        city_long_list.append(city_index_df['coord'][row]['lon'])
        city_lat_list.append(city_index_df['coord'][row]['lat'])
   
# Combine all items into a dataframe
df = pd.DataFrame({"city":city_name_list, 'id':city_id_list, 'country':city_country_list,
                   'lon':city_long_list, 'lat':city_lat_list})
df

,city,country,id,lat,lon
0,Black Hawk County,US,4848699,42.466648,-92.316849
1,Lepa,PH,1706836,15.882600,120.449303
2,Highland Park,US,4294771,38.184509,-85.753021


In [103]:
# Pick 500 random cities

random_city_df = pd.DataFrame(columns=['coord', 'country', 'id', 'name', 'lon', 'lat'])

counter = 0
total_cities = 5
while counter < total_cities:
    row = random.randint(0, len(city_index_df))
    if any(random_city_df.id == city_index_df['id'][row]):
        pass
    else:
        random_city_df.loc[counter] = city_index_df.iloc[row,:]
        random_city_df.lat[counter] = city_index_df['coord'][row]['lat']
        random_city_df.lon[counter] = city_index_df['coord'][row]['lon']
        counter = counter + 1

# Is this needed?
random_city_df["lat"] = random_city_df["lat"].map("{:.5f}".format)        
random_city_df["lon"] = random_city_df["lon"].map("{:.5f}".format)

df = random_city_df.style.format({'lat' : '{:.2f}'})


df
#random_city_df['temp'] = abs(int(random_city_df.lat))

#random_city_df

ValueError: Unknown format code 'f' for object of type 'str'

<h1>Weather API Call</h1>

In [ ]:
# Let's get the JSON for 100 posts sequentially.
url = "http://api.openweathermap.org/data/2.5/forecast?id=524901&APPID="+api_key

In [ ]:
# Make a request for each city
for x in range(total_cities):
    print(f"Request {x+1} of {total_cities}: {city_list[x]}")
    print('url')

    # Get one of the posts
    #post_response = requests.get(url)

    # Save post's JSON
    #response_json.append(post_response.json())

<h1>Print Charts</h1>

In [ ]:
# This section prints a scatter plot of latitude by temperature

# Create bubble plots
plt.scatter(latitude_list, max_temp, marker="o", facecolors='gold', edgecolors="black", alpha=1)

# Create a title, x label, and y label
plt.title("City Temperature by Temperature (F\N{DEGREE SIGN})", fontsize=20, fontweight="bold")
plt.xlabel("Latitude", fontsize=14)
plt.ylabel("Temperature (F\N{DEGREE SIGN})", fontsize=14)

#plt.colorbar()

# Set size of x and y tick labels
plt.tick_params(labelsize=12)

# Set x and y limits
#plt.xlim(0, max(rural_total_rides_by_city.max(), urban_total_rides_by_city.max(), suburban_total_rides_by_city.max())+5)
#plt.ylim(15, max(rural_avg_fare_by_city.max(), urban_avg_fare_by_city.max(), suburban_avg_fare_by_city.max())+5)

# Display bubble chart
plt.show()

<h1>Export Results</h1>